# ウイルス感染症の数理シミュレーション（基礎）: 補足資料

Julia言語で書いている

In [1]:
using DifferentialEquations
using ModelingToolkit
using Plots

# 指数方程式

In [2]:
let
    t::Vector{Float64} = Vector(0:0.001:3)
    
    G::Float64 = 1.5
    Vi_0::Float64 = 2.0 * 10^3
    Vi_t::Vector{Float64} = Vi_0 * exp.(G * t)
    t_D::Float64 = log(2) / G
    
    inc_plot = plot(
        t,
        Vi_t,
        label="V(t)",
        color="red",
        title="Increase virus",
        xlabel="Day",
        ylabel="Virus",
    )
    vline!(
        inc_plot,
        [t_D],
        color="black",
        linestyle=:dash,
        label="tD",
    )
    
    D::Float64 = 1.5
    Vd_0::Float64 = 2.0 * 10^3
    Vd_t::Vector{Float64} = Vd_0 * exp.(-D * t)
    t_H::Float64 = log(2) / D
    
    dec_plot = plot(
        t,
        Vd_t,
        label="V(t)",
        color=:blue,
        title="Decrease virus",
        xlabel="Day",
        ylabel="Virus",
    )
    vline!(
        dec_plot,
        [t_H],
        color=:black,
        linestyle=:dash,
        label="tH",
    )
    
    plot(
        inc_plot,
        dec_plot,
        size=(800, 300),
        left_margin = 20Plots.px,
        bottom_margin = 20Plots.px,
    )
end

# ロジスティック方程式

In [3]:
let
    t::Vector{Float64} = Vector(0:0.0001:0.5)
    
    function logit_eq(
        t::Vector{Float64},
        V_0::Float64;
        G::Float64=50.0,
        K_v::Float64=5.0 * 10^6,
    )
        return (
            (V_0 * K_v * exp.(G * t))
            ./ (K_v .- V_0 .+ V_0 * exp.(G * t))
        )
    end
    
    logit_plot = plot(
        t,
        [
            logit_eq(t, 1.0 * 10^7),
            logit_eq(t, 2.5 * 10^6),
            logit_eq(t, 1.0),
        ],
        title="Logistics equation",
        xlabel="Day",
        ylabel="Virus",
        label=[
            "V(0): 1.0 * 10^7" "V(0): 2.5 * 10^6" "V(0): 1.0"
        ],
        ylim=(0, 1.0 * 10 ^ 7),
    )
    
    plot(
        logit_plot,
        size=(400, 400),
        left_margin = 20Plots.px,
        bottom_margin = 20Plots.px,
    )
end

# 線形微分方程式

In [4]:
let
    t::Vector{Float64} = Vector(0:0.001:10)
    
    function liner_simultaneous_eq(
        t::Vector{Float64},
        beta::Float64,
        delta::Float64,
        p::Float64,
        c::Float64;
        T_0::Float64=3.7 * 10^5,
        I_0::Float64=1.0,
        V_0::Float64=100.0,
    )
        theta_1 = (-(delta + c) + sqrt((delta - c)^2 + 4 * beta * p * T_0)) / 2
        theta_2 = (-(delta + c) - sqrt((delta - c)^2 + 4 * beta * p * T_0)) / 2
        I_t = (
            ((beta * T_0 * V_0) - (delta + theta_2) * I_0) / (theta_1 - theta_2)
            * (exp.(theta_1 * t) - exp.(theta_2 * t))
        ) + I_0 * exp.(theta_2 * t)
        V_t = (
            ((p * I_0) - (c + theta_2) * I_0) / (theta_1 - theta_2)
            * (exp.(theta_1 * t) - exp.(theta_2 * t))
        ) + V_0 * exp.(theta_2 * t)
        return I_t, V_t
    end
    
    I_t, V_t = liner_simultaneous_eq(t, 1.0 * 10^-7, 1.0, 1.0 * 10^3, 10.0)
    liner_plot = plot(
        t,
        [I_t, V_t],
        title="Logistics equation",
        xlabel="Day",
        yscale=:log10,
        label=["infected cell (R0 > 1)" "virus (R0 > 1)"],
        color=:red,
        linestyle=[:solid :dash],
    )
    I_t, V_t = liner_simultaneous_eq(t, 1.0 * 10^-13, 1.0, 1.0 * 10^3, 10.0)
    plot!(
        liner_plot,
        t,
        [I_t, V_t],
        title="Logistics equation",
        xlabel="Day",
        yscale=:log10,
        label=["infected cell (R0 < 1)" "virus (R0 < 1)"],
        color=:blue,
        linestyle=[:solid :dash],
    )
    
    plot(
        liner_plot,
        size=(600, 400),
        left_margin = 20Plots.px,
        bottom_margin = 20Plots.px,
        legend=:outertopright,
    )
end

# 非線形微分方程式

In [5]:
let
    @variables t T(t)=6.4*10^6 I(t)=0.1 V(t)=5.0*10^4
    @parameters λ=0.0 d=0.05 β=8.6*10^-11 δ=1.75+0.05 p=3.0*10^4 c=1.93

    D = Differential(t)
    eqs = [
        D(T) ~ λ - d * T - β * T * V
        D(I) ~ β * T * V - δ * I
        D(V) ~ p * I - c * V
    ]
    @mtkbuild sys = ODESystem(eqs, t)

    tspan = (0.0, 25.0)
    prob = ODEProblem(sys, [], tspan)
    sol = solve(prob)

    cell_plot = plot(
        sol[t],
        [sol[T], sol[I]],
        title="Cell dinamics",
        xlabel="Day",
        ylabel="Cell",
        yscale=:log10,
        label=["target" "infected"],
        color=[:blue :red],
    )
    virus_plot = plot(
        sol[t],
        sol[V],
        title="Virus dinamics",
        xlabel="Day",
        ylabel="Virus",
        label="Virus",
        yscale=:log10,
        color=:black,
    )
    
    plot(
        cell_plot,
        virus_plot,
        size=(800, 400),
        left_margin = 20Plots.px,
        bottom_margin = 20Plots.px,
    )
end

┌ Warning: Independent variable t should be defined with @independent_variables t.
└ @ ModelingToolkit /Users/yuheiyasui/.julia/packages/ModelingToolkit/K8zNC/src/utils.jl:119